In [163]:
# Imports
import os
import base64
import requests
#from abc import ABC
import pandas as pd
from pandas.io.json import json_normalize
import geopandas as gpd


# Define aWhere API key and secret
api_key = os.environ.get('AWHERE_API_KEY')
api_secret = os.environ.get('AWHERE_API_SECRET')

## Base Class

In [2]:
class AWhereAPI():
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None):
        # Define authorization information
        self.api_key = api_key
        self.api_secret = api_secret
        self.base_64_encoded_secret_key = self.encode_secret_and_key(
            self.api_key, self.api_secret)
        self.auth_token = self.get_oauth_token(self.base_64_encoded_secret_key)

    def encode_secret_and_key(self, key, secret):
        """
        Docs:
            http://developer.awhere.com/api/authentication
        Returns:
            Returns the base64-encoded {key}:{secret} combination, seperated by a colon.
        """
        # Base64 Encode the Secret and Key
        key_secret = f"{key}:{secret}"

        encoded_key_secret = base64.b64encode(
            bytes(key_secret, 'utf-8')).decode('ascii')

        return encoded_key_secret

    def get_oauth_token(self, encoded_key_secret):
        """
        Demonstrates how to make a HTTP POST request to obtain an OAuth Token

        Docs:
            http://developer.awhere.com/api/authentication

        Returns:
            The access token provided by the aWhere API
        """
        # Define authorization parameters
        auth_url = 'https://api.awhere.com/oauth/token'

        auth_headers = {
            "Authorization": f"Basic {encoded_key_secret}",
            'Content-Type': 'application/x-www-form-urlencoded'
        }

        body = "grant_type=client_credentials"

        # Perform HTTP request for OAuth Token
        response = requests.post(
            auth_url, headers=auth_headers, data=body)

        # Return access token
        return response.json()['access_token']

## Sub-Classes

### Geolocation

In [69]:
class Geolocation(AWhereAPI):
    # Constructor - can be invoked by derived classes
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None, latitude=0, longitude=0):
        super(Geolocation, self).__init__(api_key, api_secret,
                                          base_64_encoded_secret_key, auth_token)
        self.latitude = latitude
        self.longitude = longitude

    def get_weather_forecast(self, latitude, longitude, start_day=None, end_day=None, offset=0, block_size=24):
        pass

### Field

In [70]:
class Field(AWhereAPI):
    pass

## Sub-Sub-Classes

In [71]:
# Sub-sub-class - make one each for forecast, norms, and observed? Then sub-sub-sub-class for soil temp and soil moisture?
class WeatherGeo(Geolocation):

    # Class variables - # Make this name more generic - for use with other classes lon_lat_cols,
    # drop_cols, mapping, etc. 
    forecast_main_lon_lat_cols = ['longitude', 'latitude']

    forecast_main_drop_cols = [
        'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units', 'dewPoint.units'
    ]

    forecast_main_mapping = {
        'startTime': 'start_time',
        'endTime': 'end_time',
        'conditionsCode': 'conditions_code',
        'conditionsText': 'conditions_text',
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.chance': 'precip_chance_%',
        'precipitation.amount': 'precip_amount_mm',
        'sky.cloudCover': 'sky_cloud_cover_%',
        'sky.sunshine': 'sky_sunshine_%',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.average': 'wind_avg_m_per_sec',
        'wind.max': 'wind_max_m_per_sec',
        'wind.min': 'wind_min_m_per_sec',
        'wind.bearing': 'wind_bearing_deg',
        'wind.direction': 'wind_direction_compass',
        'dewPoint.amount': 'dew_point_cels'
    }

    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None, latitude=0, longitude=0, _geolocation_url=None):
        super(WeatherGeo, self).__init__(api_key, api_secret,
                                         base_64_encoded_secret_key, auth_token, latitude, longitude)
        self._geolocation_url = 'https://api.awhere.com/v2/weather/locations'

    def get_weather_forecast(self, latitude, longitude, start_day=None, end_day=None, offset=0, block_size=24):
        """
        Performs a HTTP GET request to obtain the 7-day forecast.

        Docs:
            http://developer.awhere.com/api/forecast-weather-api

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response: dict
            Dictionary containing the forecast.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self._geolocation_url}/{latitude},{longitude}/forecasts?limit=10&offset={offset}&blockSize={block_size}"
        url_start_date = f"{self._geolocation_url}/{latitude},{longitude}/forecasts/{start_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_end_date = f"{self._geolocation_url}/{latitude},{longitude}/forecasts/{end_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_both_dates = f"{self._geolocation_url}/{latitude},{longitude}/forecasts/{start_day},{end_day}?limit=10&offset={offset}&blockSize={block_size}"

        # Perform the HTTP request to obtain the Forecast for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return forecast
        return response.json()

    @staticmethod # Make this name more generic - for use with other classes (Class.extract())
    def extract_forecast_main_data(forecast):
        """Extract aWhere forecast data and returns
        it in a pandas dataframe.
        """
        # Initialize lists to store forecast
        forecast_main_list = []

        # Check if more than one day
        if forecast.get('forecasts'):
            forecast_iterator = json_normalize(forecast.get('forecasts'))

        # Single day
        else:
            forecast_iterator = json_normalize(forecast)

        # Loop through each row in the top-level flattened dataframe
        for index, row in forecast_iterator.iterrows():

            # Extract date, lat, lon for insertion into lower-level dataframe outputs
            date = row['date']
            lat = row['location.latitude']
            lon = row['location.longitude']

            # Extract the main forecast from the top-level dataframe
            forecast = row['forecast']

            # Faltten data into dataframe
            forecast_norm = json_normalize(forecast)

            # Drop soil moisture and soil temperature columns
            #  (will be extracted as indivdiual dataframes)
            forecast_norm.drop(columns=[
                'soilTemperatures',
                'soilMoisture',
            ],
                axis=1, inplace=True)
            # Assign date, lat/lon to dataframe
            forecast_norm['date'] = date
            forecast_norm['latitude'] = lat
            forecast_norm['longitude'] = lon

            # Set date as index
            forecast_norm.set_index(['date'], inplace=True)

            # Add the dataframe to a list of dataframes
            forecast_main_list.append(forecast_norm)

        # Return merged lists of dataframes into a single dataframe
        return pd.concat(forecast_main_list)

    @staticmethod # Make this name more generic - for use with other classes (Class.clean())
    def clean_dataframe(df, lon_lat_cols, drop_cols, name_map):
        """Converts dataframe to geodataframe,
        drops unnecessary columns, and renames
        columns.

        Parameters
        ----------
        df : dataframe
            Input dataframe.

        lon_lat_cols : list
            List containing the column name for longitude (list[0])
            and latitude (list[1]) attributes.

        drop_cols : list (of str)
            List of column names to be dropped.

        name_map : dict
            Dictionaty mapping old columns names (keys)
            to new column names (values).

        Returns
        -------
        gdf : geodataframe
            Cleaned geodataframe.

        Example
        -------
        """
        # Define CRS (EPSG 4326) - make this a parameter?
        crs = {'init': 'epsg:4326'}

        # Rename index - possibly as option, or take care of index prior?
        #df.index.rename('date_rename', inplace=True)

        # Create copy of input dataframe; prevents altering the original
        df_copy = df.copy()

        # Convert to geodataframe
        gdf = gpd.GeoDataFrame(
            df_copy, crs=crs, geometry=gpd.points_from_xy(
                df[lon_lat_cols[0]],
                df[lon_lat_cols[1]])
        )

        # Add lat/lon columns to drop columns list
        drop_cols += lon_lat_cols

        # Drop columns
        gdf.drop(columns=drop_cols, axis=1, inplace=True)

        # Rename columns
        gdf.rename(columns=name_map, inplace=True)

        # Return cleaned up geodataframe
        return gdf

In [20]:
awhere = AWhereAPI(api_key, api_secret)

In [21]:
awhere.auth_token

'i7UbE2NiLsAg325oFSlgQnNkGkd9'

In [12]:
#help(Geolocation)

In [25]:
geo = Geolocation(api_key, api_secret)

In [29]:
geo_rmnp = Geolocation(api_key, api_secret, latitude=40, longitude=-105)

In [32]:
geo_rmnp.latitude

40

In [33]:
geo_rmnp.longitude

-105

In [45]:
# 3rd level, (2nd subclass)
weather_geo_rmnp = WeatherGeo(api_key, api_secret, latitude=40, longitude=-105)

In [46]:
weather_geo_rmnp.latitude

40

In [47]:
weather_geo_rmnp.longitude

-105

In [61]:
weather_return = weather_geo_rmnp.get_weather_forecast(
    weather_geo_rmnp.latitude, weather_geo_rmnp.longitude)

In [63]:
weather_df = WeatherGeo.extract_forecast_main_data(weather_return)

In [64]:
weather_gdf = WeatherGeo.clean_dataframe(weather_df, WeatherGeo.forecast_main_lon_lat_cols,
    WeatherGeo.forecast_main_drop_cols, WeatherGeo.forecast_main_mapping)

In [66]:
weather_df

,startTime,endTime,conditionsCode,conditionsText,temperatures.max,temperatures.min,temperatures.units,precipitation.chance,precipitation.amount,precipitation.units,...,wind.average,wind.max,wind.min,wind.units,wind.bearing,wind.direction,dewPoint.amount,dewPoint.units,latitude,longitude
date,,,,,,,,,,,,,,,,,,,,,
2020-04-10,2020-04-10T00:00:00+00:00,2020-04-10T23:59:59+00:00,D11,"Partly Cloudy, No Rain, Light Wind/Calm",22.181128,4.520203,C,0.0,0.0000,mm,...,2.914513,6.781198,0.904299,m/sec,268.4,W,-2.986565,C,40.0,-105.0
2020-04-11,2020-04-11T00:00:00+00:00,2020-04-11T23:59:59+00:00,C11,"Mostly Clear, No Rain, Light Wind/Calm",20.049999,9.119950,C,0.0,0.0000,mm,...,4.177402,6.383487,2.569230,m/sec,260.9,W,-6.431584,C,40.0,-105.0
2020-04-12,2020-04-12T00:00:00+00:00,2020-04-12T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",18.609928,-4.244983,C,100.0,10.0000,mm,...,7.154848,13.880400,3.887463,m/sec,9.7,N,-3.640049,C,40.0,-105.0
2020-04-13,2020-04-13T00:00:00+00:00,2020-04-13T23:59:59+00:00,E31,"Mostly Cloudy, Moderate Rain, Light Wind/Calm",2.266663,-10.444544,C,100.0,3.6875,mm,...,2.392039,4.549304,0.098144,m/sec,98.0,E,-8.442663,C,40.0,-105.0
2020-04-14,2020-04-14T00:00:00+00:00,2020-04-14T23:59:59+00:00,E21,"Mostly Cloudy, Light Rain, Light Wind/Calm",4.954069,-6.948340,C,87.5,1.9375,mm,...,1.988982,3.448789,0.484934,m/sec,56.3,ENE,-5.875406,C,40.0,-105.0
2020-04-15,2020-04-15T00:00:00+00:00,2020-04-15T23:59:59+00:00,D11,"Partly Cloudy, No Rain, Light Wind/Calm",11.238805,-3.194625,C,0.0,0.0000,mm,...,2.294561,3.665068,1.333319,m/sec,325.9,NW,-8.075027,C,40.0,-105.0
2020-04-16,2020-04-16T00:00:00+00:00,2020-04-16T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",10.824980,-2.736865,C,65.0,11.0625,mm,...,2.850829,7.192179,1.112028,m/sec,107.5,ESE,-2.044650,C,40.0,-105.0
2020-04-17,2020-04-17T00:00:00+00:00,2020-04-17T23:59:59+00:00,E11,"Mostly Cloudy, No Rain, Light Wind/Calm",18.213123,-0.045308,C,0.0,0.0000,mm,...,2.541268,6.167961,0.325889,m/sec,248.2,WSW,-3.010257,C,40.0,-105.0
2020-04-18,2020-04-18T00:00:00+00:00,2020-04-18T23:59:59+00:00,D11,"Partly Cloudy, No Rain, Light Wind/Calm",22.484587,5.012423,C,0.0,0.0000,mm,...,3.748102,7.378313,1.776269,m/sec,259.6,W,-0.767444,C,40.0,-105.0


In [65]:
weather_gdf

,start_time,end_time,conditions_code,conditions_text,temp_max_cels,temp_min_cels,precip_chance_%,precip_amount_mm,sky_cloud_cover_%,sky_sunshine_%,...,rel_humidity_avg_%,rel_humidity_max_%,rel_humidity_min_%,wind_avg_m_per_sec,wind_max_m_per_sec,wind_min_m_per_sec,wind_bearing_deg,wind_direction_compass,dew_point_cels,geometry
date,,,,,,,,,,,,,,,,,,,,,
2020-04-10,2020-04-10T00:00:00+00:00,2020-04-10T23:59:59+00:00,D11,"Partly Cloudy, No Rain, Light Wind/Calm",22.181128,4.520203,0.0,0.0000,41.250,58.750,...,40.545833,63.900002,11.100000,2.914513,6.781198,0.904299,268.4,W,-2.986565,POINT (-105.00000 40.00000)
2020-04-11,2020-04-11T00:00:00+00:00,2020-04-11T23:59:59+00:00,C11,"Mostly Clear, No Rain, Light Wind/Calm",20.049999,9.119950,0.0,0.0000,36.750,63.250,...,24.050000,34.900002,11.900000,4.177402,6.383487,2.569230,260.9,W,-6.431584,POINT (-105.00000 40.00000)
2020-04-12,2020-04-12T00:00:00+00:00,2020-04-12T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",18.609928,-4.244983,100.0,10.0000,96.000,4.000,...,68.595833,91.699997,16.700001,7.154848,13.880400,3.887463,9.7,N,-3.640049,POINT (-105.00000 40.00000)
2020-04-13,2020-04-13T00:00:00+00:00,2020-04-13T23:59:59+00:00,E31,"Mostly Cloudy, Moderate Rain, Light Wind/Calm",2.266663,-10.444544,100.0,3.6875,75.500,24.500,...,75.741667,94.199997,55.799999,2.392039,4.549304,0.098144,98.0,E,-8.442663,POINT (-105.00000 40.00000)
2020-04-14,2020-04-14T00:00:00+00:00,2020-04-14T23:59:59+00:00,E21,"Mostly Cloudy, Light Rain, Light Wind/Calm",4.954069,-6.948340,87.5,1.9375,76.625,23.375,...,75.325000,94.199997,41.700001,1.988982,3.448789,0.484934,56.3,ENE,-5.875406,POINT (-105.00000 40.00000)
2020-04-15,2020-04-15T00:00:00+00:00,2020-04-15T23:59:59+00:00,D11,"Partly Cloudy, No Rain, Light Wind/Calm",11.238805,-3.194625,0.0,0.0000,48.750,51.250,...,47.570834,70.599998,27.200001,2.294561,3.665068,1.333319,325.9,NW,-8.075027,POINT (-105.00000 40.00000)
2020-04-16,2020-04-16T00:00:00+00:00,2020-04-16T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",10.824980,-2.736865,65.0,11.0625,94.000,6.000,...,80.066667,96.000000,42.566666,2.850829,7.192179,1.112028,107.5,ESE,-2.044650,POINT (-105.00000 40.00000)
2020-04-17,2020-04-17T00:00:00+00:00,2020-04-17T23:59:59+00:00,E11,"Mostly Cloudy, No Rain, Light Wind/Calm",18.213123,-0.045308,0.0,0.0000,72.375,27.625,...,53.983334,80.300003,28.299999,2.541268,6.167961,0.325889,248.2,WSW,-3.010257,POINT (-105.00000 40.00000)
2020-04-18,2020-04-18T00:00:00+00:00,2020-04-18T23:59:59+00:00,D11,"Partly Cloudy, No Rain, Light Wind/Calm",22.484587,5.012423,0.0,0.0000,56.250,43.750,...,40.204167,60.200001,22.200001,3.748102,7.378313,1.776269,259.6,W,-0.767444,POINT (-105.00000 40.00000)


# Base Classs - AWhereAPI * Good as of 4/11 (make this base class?)

In [197]:
class AWhereAPI():
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None):
        # Define authorization information
        self.api_key = api_key
        self.api_secret = api_secret
        self.base_64_encoded_secret_key = self.encode_secret_and_key(
            self.api_key, self.api_secret)
        self.auth_token = self.get_oauth_token(self.base_64_encoded_secret_key)

    def encode_secret_and_key(self, key, secret):
        """
        Docs:
            http://developer.awhere.com/api/authentication
        Returns:
            Returns the base64-encoded {key}:{secret} combination, seperated by a colon.
        """
        # Base64 Encode the Secret and Key
        key_secret = f"{key}:{secret}"

        encoded_key_secret = base64.b64encode(
            bytes(key_secret, 'utf-8')).decode('ascii')

        return encoded_key_secret

    def get_oauth_token(self, encoded_key_secret):
        """
        Demonstrates how to make a HTTP POST request to obtain an OAuth Token

        Docs:
            http://developer.awhere.com/api/authentication

        Returns:
            The access token provided by the aWhere API
        """
        # Define authorization parameters
        auth_url = 'https://api.awhere.com/oauth/token'

        auth_headers = {
            "Authorization": f"Basic {encoded_key_secret}",
            'Content-Type': 'application/x-www-form-urlencoded'
        }

        body = "grant_type=client_credentials"

        # Perform HTTP request for OAuth Token
        response = requests.post(
            auth_url, headers=auth_headers, data=body)

        # Return access token
        return response.json()['access_token']

# Subclass - Weather - Good as of 4/11 (make this abstract class?)

In [198]:
class Weather(AWhereAPI):
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None, api_url=None):
        super(Weather, self).__init__(api_key, api_secret,
                                      base_64_encoded_secret_key, auth_token)

        self.api_url = 'https://api.awhere.com/v2/weather'

    @staticmethod
    def get_data():
        pass
        
    @staticmethod
    def extract_data():
        pass

    @staticmethod
    def clean_data(df, lon_lat_cols, drop_cols, name_map):
        """Converts dataframe to geodataframe,
        drops unnecessary columns, and renames
        columns.

        Parameters
        ----------
        df : dataframe
            Input dataframe.

        lon_lat_cols : list
            List containing the column name for longitude (list[0])
            and latitude (list[1]) attributes.

        drop_cols : list (of str)
            List of column names to be dropped.

        name_map : dict
            Dictionaty mapping old columns names (keys)
            to new column names (values).

        Returns
        -------
        gdf : geodataframe
            Cleaned geodataframe.

        Example
        -------
        """
        # Define CRS (EPSG 4326) - make this a parameter?
        crs = {'init': 'epsg:4326'}

        # Rename index - possibly as option, or take care of index prior?
        #df.index.rename('date_rename', inplace=True)

        # Create copy of input dataframe; prevents altering the original
        df_copy = df.copy()

        # Convert to geodataframe
        gdf = gpd.GeoDataFrame(
            df_copy, crs=crs, geometry=gpd.points_from_xy(
                df[lon_lat_cols[0]],
                df[lon_lat_cols[1]])
        )

        # Add lat/lon columns to drop columns list
        drop_cols += lon_lat_cols

        # Drop columns
        gdf.drop(columns=drop_cols, axis=1, inplace=True)

        # Rename columns
        gdf.rename(columns=name_map, inplace=True)

        # Return cleaned up geodataframe
        return gdf

In [100]:
test_weather = Weather(api_key, api_secret)

In [102]:
test_weather.api_url

'https://api.awhere.com/v2/weather'

In [11]:
print(test_weather.longitude)

-105


In [12]:
#print(test_weather._api_url)

## Sub-sub-class - WeatherLocation * Good as of 4/11 (make this abstract class?)

In [199]:
# Sub-sub-class - make one each for forecast, norms, and observed? Then sub-sub-sub-class for soil temp and soil moisture?
class WeatherLocation(Weather):

    def __init__(self, api_key, api_secret,
                 base_64_encoded_secret_key=None, auth_token=None, api_url=None):

        super(WeatherLocation, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.api_url = f"{self.api_url}/locations"

In [183]:
default_test = WeatherLocation(api_key, api_secret)

In [184]:
#default_test.latitude

In [185]:
#default_test.longitude

In [186]:
default_test.api_url

'https://api.awhere.com/v2/weather/locations'

## Sub-sub-class - WeatherField

In [13]:
# # Sub-sub-class - make one each for forecast, norms, and observed? Then sub-sub-sub-class for soil temp and soil moisture?
# class WeatherNorms(Weather):

#     def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
#                  auth_token=None, latitude=0, longitude=0): #, _api_url=None):

#         super(WeatherNorms, self).__init__(
#             api_key, api_secret, base_64_encoded_secret_key, auth_token, latitude, longitude) #, _api_url)

#         #self._api_url = _api_url

#         #def get_norms():
#          #   pass

## Sub-sub-sub-class - WeatherLocationNorms ** Good as of 4/11

In [200]:
# Sub-sub-class - make one each for forecast, norms, and observed? Then sub-sub-sub-class for soil temp and soil moisture?
class WeatherLocationNorms(WeatherLocation):

    # Class variables for clean_data() function
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'meanTemp.units', 'maxTemp.units',
        'minTemp.units', 'precipitation.units', 'solar.units',
        'dailyMaxWind.units', 'averageWind.units'
    ]

    rename_map = {
        'meanTemp.average': 'mean_temp_avg_cels',
        'meanTemp.stdDev': 'mean_temp_std_dev_cels',
        'maxTemp.average': 'max_temp_avg_cels',
        'maxTemp.stdDev': 'max_temp_std_dev_cels',
        'minTemp.average': 'min_temp_avg_cels',
        'minTemp.stdDev': 'min_temp_std_dev_cels',
        'precipitation.average': 'precip_avg_mm',
        'precipitation.stdDev': 'precip_std_dev_mm',
        'solar.average': 'solar_avg_w_h_per_m2',
        'minHumidity.average': 'min_humiduty_avg_%',
        'minHumidity.stdDev': 'min_humidity_std_dev_%',
        'maxHumidity.average': 'max_humiduty_avg_%',
        'maxHumidity.stdDev': 'max_humidity_std_dev_%',
        'dailyMaxWind.average': 'daily_max_wind_avg_m_per_sec',
        'dailyMaxWind.stdDev': 'daily_max_wind_std_dev_m_per_sec',
        'averageWind.average': 'average_wind_m_per_sec',
        'averageWind.stdDev': 'average_wind_std_dev_m_per_sec'
    }

    def __init__(self, api_key, api_secret, latitude, longitude, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(WeatherLocationNorms, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude 
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/norms"

    def get_data(self, latitude, longitude, start_day, end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 10-year historical norms.

        Docs:
            http://developer.awhere.com/api/reference/weather/norms

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the norms.

        Example
        -------
        """
        """# Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        response = requests.get(
            f"{self._weather_url}/{field_id}/norms/{start_day}",
            headers=auth_headers)

        # Return the norms
        return response.json()"""

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self.api_url}/{start_day}?limit=10&offset={offset}"
        url_multiple_days = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(historic_norms):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere historic norm data.

        Handles both single-day norms and multiple days.

        Parameters
        ----------
        historic_norms : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        historic_norms_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in norms
        if historic_norms.get('norms'):
            # Flatten to dataframe
            historic_norms_df = json_normalize(historic_norms.get('norms'))

        # Single-day norm
        else:
            # Flatten to dataframe
            historic_norms_df = json_normalize(historic_norms)

        # Set day as index
        historic_norms_df.set_index('day', inplace=True)

        # Drop unnecessary columns
        historic_norms_df.drop(
            columns=['_links.self.href'],
            axis=1, inplace=True)

        # Return dataframe
        return historic_norms_df

In [201]:
test_weather_norms = WeatherLocationNorms(
    api_key, api_secret, latitude=40.31324, longitude=-105.648222)

In [202]:
test_weather_norms.api_url

'https://api.awhere.com/v2/weather/locations/40.31324,-105.648222/norms'

In [203]:
test_weather_norms.get_data(
    test_weather_norms.latitude, test_weather_norms.longitude,
    '05-05', '05-15')

{'norms': [{'day': '05-05',
   'location': {'latitude': 40.31324, 'longitude': -105.648222},
   'meanTemp': {'average': 4.755999994277954,
    'stdDev': 2.98513703471686,
    'units': 'C'},
   'maxTemp': {'average': 12.612999963760377,
    'stdDev': 4.306328748584195,
    'units': 'C'},
   'minTemp': {'average': -3.1009999752044677,
    'stdDev': 2.6286729300852723,
    'units': 'C'},
   'precipitation': {'average': 0.23410000801086425,
    'stdDev': 0.7402892255779947,
    'units': 'mm'},
   'solar': {'average': 6617.703344726562,
    'stdDev': 1489.9750609856167,
    'units': 'Wh/m^2'},
   'minHumidity': {'average': 23.203999710083007, 'stdDev': 17.4302041376372},
   'maxHumidity': {'average': 78.59899978637695, 'stdDev': 12.711234081403106},
   'dailyMaxWind': {'average': 7.25918231010437,
    'stdDev': 2.18033585092304,
    'units': 'm/sec'},
   'averageWind': {'average': 3.262729787826538,
    'stdDev': 1.052785350892358,
    'units': 'm/sec'},
   '_links': {'self': {'href': '/v2/

In [192]:
norms_json = test_weather_norms.get_data(
    test_weather_norms.latitude, test_weather_norms.longitude,
    '05-05', '05-15')

In [204]:
df = WeatherLocationNorms.extract_data(norms_json)

In [205]:
df

,location.latitude,location.longitude,meanTemp.average,meanTemp.stdDev,meanTemp.units,maxTemp.average,maxTemp.stdDev,maxTemp.units,minTemp.average,minTemp.stdDev,...,minHumidity.average,minHumidity.stdDev,maxHumidity.average,maxHumidity.stdDev,dailyMaxWind.average,dailyMaxWind.stdDev,dailyMaxWind.units,averageWind.average,averageWind.stdDev,averageWind.units
day,,,,,,,,,,,,,,,,,,,,,
05-05,40.0,-105.0,13.9460,3.890321,C,22.083,5.551672,C,5.809,3.525650,...,22.454000,22.333358,70.451000,13.848074,6.584205,2.124374,m/sec,2.795455,0.892515,m/sec
05-06,40.0,-105.0,14.4150,3.582551,C,21.849,5.164409,C,6.981,2.498215,...,26.088000,14.494682,73.768000,18.025834,7.038779,1.919636,m/sec,2.866111,0.954224,m/sec
05-07,40.0,-105.0,12.3885,4.763780,C,18.574,6.989987,C,6.203,3.236504,...,39.069000,24.087735,84.339999,12.904113,7.047276,2.314907,m/sec,2.817376,1.084698,m/sec
05-08,40.0,-105.0,11.2900,4.472330,C,17.520,6.252500,C,5.060,3.502110,...,42.004001,24.428537,84.154999,18.977546,6.414071,1.628986,m/sec,2.930409,0.871596,m/sec
05-09,40.0,-105.0,11.8510,5.320620,C,18.739,7.408928,C,4.963,4.130421,...,38.504000,24.690818,81.045999,21.408004,7.503801,2.915196,m/sec,3.022484,1.338749,m/sec
05-10,40.0,-105.0,12.2765,4.953243,C,19.084,7.114010,C,5.469,3.454776,...,35.065001,22.852383,84.315001,15.293012,7.565571,2.135641,m/sec,3.154584,0.967800,m/sec
05-11,40.0,-105.0,9.4190,3.781469,C,15.256,4.823213,C,3.582,3.136076,...,48.092000,20.525045,93.670999,4.222307,6.967221,1.404956,m/sec,3.357547,1.342111,m/sec
05-12,40.0,-105.0,10.2100,5.299581,C,16.425,8.207957,C,3.995,3.466565,...,43.446000,23.790209,92.188000,5.866668,5.401922,1.219847,m/sec,2.604031,1.039167,m/sec
05-13,40.0,-105.0,12.7715,5.117035,C,19.418,6.659005,C,6.125,4.124823,...,32.739000,15.429507,84.268000,13.367423,6.015931,1.993536,m/sec,2.445945,0.753984,m/sec


In [206]:
gdf = Weather.clean_data(
    df, 
    WeatherLocationNorms.coord_cols,
    WeatherLocationNorms.drop_cols, 
    WeatherLocationNorms.rename_map)

In [207]:
gdf

,mean_temp_avg_cels,mean_temp_std_dev_cels,max_temp_avg_cels,max_temp_std_dev_cels,min_temp_avg_cels,min_temp_std_dev_cels,precip_avg_mm,precip_std_dev_mm,solar_avg_w_h_per_m2,solar.stdDev,min_humiduty_avg_%,min_humidity_std_dev_%,max_humiduty_avg_%,max_humidity_std_dev_%,daily_max_wind_avg_m_per_sec,daily_max_wind_std_dev_m_per_sec,average_wind_m_per_sec,average_wind_std_dev_m_per_sec,geometry
day,,,,,,,,,,,,,,,,,,,
05-05,13.9460,3.890321,22.083,5.551672,5.809,3.525650,2.605778,4.210784,6363.582080,1534.962877,22.454000,22.333358,70.451000,13.848074,6.584205,2.124374,2.795455,0.892515,POINT (-105.00000 40.00000)
05-06,14.4150,3.582551,21.849,5.164409,6.981,2.498215,4.613071,7.772575,5644.947485,1102.703641,26.088000,14.494682,73.768000,18.025834,7.038779,1.919636,2.866111,0.954224,POINT (-105.00000 40.00000)
05-07,12.3885,4.763780,18.574,6.989987,6.203,3.236504,6.720669,4.666867,4555.624194,1505.715592,39.069000,24.087735,84.339999,12.904113,7.047276,2.314907,2.817376,1.084698,POINT (-105.00000 40.00000)
05-08,11.2900,4.472330,17.520,6.252500,5.060,3.502110,5.545790,6.694246,4932.714453,2058.613223,42.004001,24.428537,84.154999,18.977546,6.414071,1.628986,2.930409,0.871596,POINT (-105.00000 40.00000)
05-09,11.8510,5.320620,18.739,7.408928,4.963,4.130421,7.477840,10.010236,5319.268359,1856.340854,38.504000,24.690818,81.045999,21.408004,7.503801,2.915196,3.022484,1.338749,POINT (-105.00000 40.00000)
05-10,12.2765,4.953243,19.084,7.114010,5.469,3.454776,0.914930,2.893263,5171.516089,1567.909975,35.065001,22.852383,84.315001,15.293012,7.565571,2.135641,3.154584,0.967800,POINT (-105.00000 40.00000)
05-11,9.4190,3.781469,15.256,4.823213,3.582,3.136076,3.592210,10.552066,4608.711572,1950.805971,48.092000,20.525045,93.670999,4.222307,6.967221,1.404956,3.357547,1.342111,POINT (-105.00000 40.00000)
05-12,10.2100,5.299581,16.425,8.207957,3.995,3.466565,1.375610,2.465522,5103.596338,2413.153494,43.446000,23.790209,92.188000,5.866668,5.401922,1.219847,2.604031,1.039167,POINT (-105.00000 40.00000)
05-13,12.7715,5.117035,19.418,6.659005,6.125,4.124823,0.583680,1.283641,5422.280029,1653.875556,32.739000,15.429507,84.268000,13.367423,6.015931,1.993536,2.445945,0.753984,POINT (-105.00000 40.00000)


## Sub-sub-sub-class - WeatherLocationObserved * Good 4/11

In [208]:
class WeatherLocationObserved(WeatherLocation):

    # Class variables for clean_data() function
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units'
    ]

    rename_map = {
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.amount': 'precip_amount_mm',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.morningMax': 'wind_morning_max_m_per_sec',
        'wind.dayMax': 'wind_day_max_m_per_sec',
        'wind.average': 'wind_avg_m_per_sec',
    }

    def __init__(self, api_key, api_secret, latitude, longitude,
                 base_64_encoded_secret_key=None, auth_token=None, api_url=None):

        super(WeatherLocationObserved, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/observations"

    def get_data(self, latitude, longitude, start_day=None, end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 7-day observed weather.

        Docs:
            http://developer.awhere.com/api/reference/weather/observations

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the observed weather.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self.api_url}?limit=10&offset={offset}"
        url_start_date = f"{self.api_url}/{start_day}"
        url_end_date = f"{self.api_url}/{end_day}"
        url_both_dates = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Perform the HTTP request to obtain the norms for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return the observed
        return response.json()

    @staticmethod
    def extract_data(observed_weather):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere observed weather data.

        Parameters
        ----------
        observed_weather : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        observed_weather_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in observed
        if observed_weather.get('observations'):
            # Flatten to dataframe
            observed_weather_df = json_normalize(
                observed_weather.get('observations'))

        # Single-day observed
        else:
            # Flatten to dataframe
            observed_weather_df = json_normalize(observed_weather)

        # Set date as index
        observed_weather_df.set_index('date', inplace=True)

        # Drop unnecessary columns
        observed_weather_df.drop(
            columns=['_links.self.href'],
            axis=1, inplace=True)

        # Return dataframe
        return observed_weather_df

In [213]:
observed = WeatherLocationObserved(api_key, api_secret, latitude=42.5, longitude=-72.5)

In [219]:
json_return = observed.get_data(observed.latitude, observed.longitude, start_day='04-07')

In [209]:
#observed_loc_return.get_observed(observed_loc_return.latitude, observed_loc_return.longitude)

In [220]:
df = WeatherLocationObserved.extract_data(json_return)

In [221]:
df

,location.latitude,location.longitude,temperatures.max,temperatures.min,temperatures.units,precipitation.amount,precipitation.units,solar.amount,solar.units,relativeHumidity.max,relativeHumidity.min,wind.morningMax,wind.dayMax,wind.average,wind.units
date,,,,,,,,,,,,,,,
2020-04-07,42.5,-72.5,16.950001,2.09,C,0.0,mm,6011.960938,Wh/m^2,68.730003,19.9,5.982268,6.863383,2.625634,m/sec


In [226]:
gdf = Weather.clean_data( # Can also to WeatherLocationObserved.clean_data()
    df,
    WeatherLocationObserved.coord_cols,
    WeatherLocationObserved.drop_cols,
    WeatherLocationObserved.rename_map
)

In [227]:
gdf

,temp_max_cels,temp_min_cels,precip_amount_mm,solar_energy_w_h_per_m2,rel_humidity_max_%,rel_humidity_min_%,wind_morning_max_m_per_sec,wind_day_max_m_per_sec,wind_avg_m_per_sec,geometry
date,,,,,,,,,,
2020-04-07,16.950001,2.09,0.0,6011.960938,68.730003,19.9,5.982268,6.863383,2.625634,POINT (-72.50000 42.50000)


## Sub-sub-sub-class - WeatherLocationForecast * Good 4/11

In [242]:
class WeatherLocationForecast(WeatherLocation):

    # Class variables for clean_data() function
    # Main forecast
    coord_cols = ['longitude', 'latitude']

    drop_cols = [
        'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units', 'dewPoint.units'
    ]

    rename_map = {
        'startTime': 'start_time',
        'endTime': 'end_time',
        'conditionsCode': 'conditions_code',
        'conditionsText': 'conditions_text',
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.chance': 'precip_chance_%',
        'precipitation.amount': 'precip_amount_mm',
        'sky.cloudCover': 'sky_cloud_cover_%',
        'sky.sunshine': 'sky_sunshine_%',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.average': 'wind_avg_m_per_sec',
        'wind.max': 'wind_max_m_per_sec',
        'wind.min': 'wind_min_m_per_sec',
        'wind.bearing': 'wind_bearing_deg',
        'wind.direction': 'wind_direction_compass',
        'dewPoint.amount': 'dew_point_cels'
    }
    
    # Soil
    soil_coord_cols = ['longitude', 'latitude']

    soil_drop_cols = ['units']

    soil_rename_map = {
        'average_temp': 'soil_temp_avg_cels',
        'max_temp': 'soil_temp_max_cels',
        'min_temp': 'soil_temp_min_cels',
        'average_moisture': 'soil_moisture_avg_%',
        'max_moisture': 'soil_moisture_max_%',
        'min_moisture': 'soil_moisture_min_%',
    }

    def __init__(self, api_key, api_secret, latitude, longitude, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(WeatherLocationForecast, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude 
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/forecasts"

    def get_data(self, latitude, longitude, start_day=None, end_day=None, offset=0, block_size=24):
        """
        Performs a HTTP GET request to obtain the 7-day forecast.

        Docs:
            http://developer.awhere.com/api/forecast-weather-api

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response: dict
            Dictionary containing the forecast.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self.api_url}?limit=10&offset={offset}&blockSize={block_size}"
        url_start_date = f"{self.api_url}/{start_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_end_date = f"{self.api_url}/{end_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_both_dates = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}&blockSize={block_size}"

        # Perform the HTTP request to obtain the Forecast for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return forecast
        return response.json()
    
    @staticmethod
    def extract_data(forecast):
        """Extract aWhere forecast data and returns
        it in a pandas dataframe.
        """
        # Initialize lists to store forecast
        forecast_main_list = []

        # Check if more than one day
        if forecast.get('forecasts'):
            forecast_iterator = json_normalize(forecast.get('forecasts'))

        # Single day
        else:
            forecast_iterator = json_normalize(forecast)

        # Loop through each row in the top-level flattened dataframe
        for index, row in forecast_iterator.iterrows():

            # Extract date, lat, lon for insertion into lower-level dataframe outputs
            date = row['date']
            lat = row['location.latitude']
            lon = row['location.longitude']

            # Extract the main forecast from the top-level dataframe
            forecast = row['forecast']

            # Faltten data into dataframe
            forecast_norm = json_normalize(forecast)

            # Drop soil moisture and soil temperature columns
            #  (will be extracted as indivdiual dataframes)
            forecast_norm.drop(columns=[
                'soilTemperatures',
                'soilMoisture',
            ],
                axis=1, inplace=True)
            # Assign date, lat/lon to dataframe
            forecast_norm['date'] = date
            forecast_norm['latitude'] = lat
            forecast_norm['longitude'] = lon

            # Set date as index
            forecast_norm.set_index(['date'], inplace=True)

            # Add the dataframe to a list of dataframes
            forecast_main_list.append(forecast_norm)

        # Return merged lists of dataframes into a single dataframe
        return pd.concat(forecast_main_list)
    
    @staticmethod
    def extract_soil(forecast):
        """Extract aWhere forecast soil
        data and returns it in a pandas dataframe.
        """
        # Initialize lists to store soil dataframes
        forecast_soil_list = []

        # Check if more than one day
        if forecast.get('forecasts'):
            forecast_iterator = json_normalize(forecast.get('forecasts'))

        # Single day
        else:
            forecast_iterator = json_normalize(forecast)

        # Loop through each row in the top-level flattened dataframe
        for index, row in forecast_iterator.iterrows():

            # Extract date, lat, lon for insertion into lower-level dataframe outputs
            date = row['date']
            lat = row['location.latitude']
            lon = row['location.longitude']

            # Get soil temperature data
            forecast_soil_temp = row['forecast'][0].get('soilTemperatures')
            forecast_soil_moisture = row['forecast'][0].get('soilMoisture')

            # Flatten data into dataframe
            forecast_soil_temp_df = json_normalize(forecast_soil_temp)
            forecast_soil_moisture_df = json_normalize(forecast_soil_moisture)

            # Combine temperature and moisture
            forecast_soil_df = forecast_soil_temp_df.merge(
                forecast_soil_moisture_df, on='depth', suffixes=('_temp', '_moisture'))

            # Assign date, lat/lon to dataframe
            forecast_soil_df['date'] = date
            forecast_soil_df['latitude'] = lat
            forecast_soil_df['longitude'] = lon

            # Shorten depth values to numerics (will be used in MultiIndex)
            forecast_soil_df['depth'] = forecast_soil_df['depth'].apply(lambda x: x[0:-15])

            # Rename depth prior to indexing
            forecast_soil_df.rename(columns={'depth': 'ground_depth_m'}, inplace=True)

            # Create multi-index dataframe for date and soil depth (rename depth columns? rather long)
            soil_multi_index = forecast_soil_df.set_index(
                ['date', 'ground_depth_m'])

            # Add dataframe to list of dataframes
            forecast_soil_list.append(soil_multi_index)

        # Return merged lists of dataframes into a single dataframe
        return pd.concat(forecast_soil_list)

In [243]:
forecast = WeatherLocationForecast(
    api_key, api_secret, latitude=40.31324, longitude=-105.648222)

In [233]:
forecast.api_url

'https://api.awhere.com/v2/weather/locations/40.31324,-105.648222/forecasts'

In [245]:
# Can remove lat/lon from parameters of get data, since URLs already have them from API URL
json_return = forecast.get_data(
    forecast.latitude, forecast.longitude, start_day='2020-04-13', end_day='2020-04-15')

In [246]:
df_main = WeatherLocationForecast.extract_data(json_return)

In [247]:
df_main

,startTime,endTime,conditionsCode,conditionsText,temperatures.max,temperatures.min,temperatures.units,precipitation.chance,precipitation.amount,precipitation.units,...,wind.average,wind.max,wind.min,wind.units,wind.bearing,wind.direction,dewPoint.amount,dewPoint.units,latitude,longitude
date,,,,,,,,,,,,,,,,,,,,,
2020-04-13,2020-04-13T00:00:00+00:00,2020-04-13T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",-6.769336,-15.606348,C,100.0,8.6250,mm,...,3.199309,7.980752,0.565157,m/sec,260.5,W,-13.520109,C,40.31324,-105.648224
2020-04-14,2020-04-14T00:00:00+00:00,2020-04-14T23:59:59+00:00,F21,"Cloudy, Light Rain, Light Wind/Calm",-7.383305,-19.306795,C,100.0,2.1875,mm,...,3.985835,9.068513,0.358080,m/sec,291.2,WNW,-16.119708,C,40.31324,-105.648224
2020-04-15,2020-04-15T00:00:00+00:00,2020-04-15T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",-4.566671,-12.347621,C,100.0,11.1250,mm,...,3.846708,5.642847,2.396491,m/sec,269.1,W,-9.991396,C,40.31324,-105.648224


In [249]:
gdf_main = Weather.clean_data(
    df_main,
    WeatherLocationForecast.coord_cols,
    WeatherLocationForecast.drop_cols,
    WeatherLocationForecast.rename_map    
)

In [250]:
gdf

,start_time,end_time,conditions_code,conditions_text,temp_max_cels,temp_min_cels,precip_chance_%,precip_amount_mm,sky_cloud_cover_%,sky_sunshine_%,...,rel_humidity_avg_%,rel_humidity_max_%,rel_humidity_min_%,wind_avg_m_per_sec,wind_max_m_per_sec,wind_min_m_per_sec,wind_bearing_deg,wind_direction_compass,dew_point_cels,geometry
date,,,,,,,,,,,,,,,,,,,,,
2020-04-13,2020-04-13T00:00:00+00:00,2020-04-13T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",-6.769336,-15.606348,100.0,8.6250,99.75,0.25,...,85.408333,97.300003,69.599998,3.199309,7.980752,0.565157,260.5,W,-13.520109,POINT (-105.64822 40.31324)
2020-04-14,2020-04-14T00:00:00+00:00,2020-04-14T23:59:59+00:00,F21,"Cloudy, Light Rain, Light Wind/Calm",-7.383305,-19.306795,100.0,2.1875,87.75,12.25,...,83.541666,94.800003,69.500000,3.985835,9.068513,0.358080,291.2,WNW,-16.119708,POINT (-105.64822 40.31324)
2020-04-15,2020-04-15T00:00:00+00:00,2020-04-15T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",-4.566671,-12.347621,100.0,11.1250,100.00,0.00,...,91.295833,98.199997,76.199997,3.846708,5.642847,2.396491,269.1,W,-9.991396,POINT (-105.64822 40.31324)


In [251]:
df_soil = WeatherLocationForecast.extract_soil(json_return)

In [253]:
df_soil

average_temp  max_temp   min_temp units  \
date       ground_depth_m                                            
2020-04-13 0-0.1              -4.855453 -1.131266  -6.701929     C   
           0.1-0.4            -1.536125 -1.530000  -1.540000     C   
           0.4-1              -2.233402 -2.187637  -2.290977     C   
           1-2                -1.247825 -1.241575  -1.248242     C   
2020-04-14 0-0.1              -9.531484 -6.437191 -11.479395     C   
           0.1-0.4            -1.572594 -1.537637  -1.740742     C   
           0.4-1              -2.140139 -2.097637  -2.184303     C   
           1-2                -1.249243 -1.248828  -1.250000     C   
2020-04-15 0-0.1              -8.798222 -7.131714  -9.331738     C   
           0.1-0.4            -2.081960 -1.774329  -2.317500     C   
           0.4-1              -2.064796 -2.047637  -2.094303     C   
           1-2                -1.248267 -1.248242  -1.248633     C   

                           average_moisture  max_moisture  min_moisture  \
date       ground_depth_m                                                 
2020-04-13 0-0.1                  36.912499     37.200001     36.633335   
           0.1-0.4                35.874406     36.011906     35.711906   
           0.4-1                  26.103708     26.103708     26.103708   
           1-2                    19.343128     19.343128     19.343128   
2020-04-14 0-0.1                  37.454167     37.700001     37.200001   
           0.1-0.4                35.620240     35.711906     35.511906   
           0.4-1                  26.103708     26.103708     26.103708   
           1-2                    19.343128     19.343128     19.343128   
2020-04-15 0-0.1                  37.816667     37.900002     37.700001   
           0.1-0.4                35.466073     35.511906     35.411907   
           0.4-1                  26.103708     26.103708     26.103708   
           1-2                    19.343128     19.343128     19.343128   

                           latitude   longitude  
date       ground_depth_m                        
2020-04-13 0-0.1           40.31324 -105.648224  
           0.1-0.4         40.31324 -105.648224  
           0.4-1           40.31324 -105.648224  
           1-2             40.31324 -105.648224  
2020-04-14 0-0.1           40.31324 -105.648224  
           0.1-0.4         40.31324 -105.648224  
           0.4-1           40.31324 -105.648224  
           1-2             40.31324 -105.648224  
2020-04-15 0-0.1           40.31324 -105.648224  
           0.1-0.4         40.31324 -105.648224  
           0.4-1           40.31324 -105.648224  
           1-2             40.31324 -105.648224

In [254]:
gdf_soil = Weather.clean_data(
    df_soil,
    WeatherLocationForecast.soil_coord_cols,
    WeatherLocationForecast.soil_drop_cols,
    WeatherLocationForecast.soil_rename_map
)

In [255]:
gdf_soil

soil_temp_avg_cels  soil_temp_max_cels  \
date       ground_depth_m                                           
2020-04-13 0-0.1                    -4.855453           -1.131266   
           0.1-0.4                  -1.536125           -1.530000   
           0.4-1                    -2.233402           -2.187637   
           1-2                      -1.247825           -1.241575   
2020-04-14 0-0.1                    -9.531484           -6.437191   
           0.1-0.4                  -1.572594           -1.537637   
           0.4-1                    -2.140139           -2.097637   
           1-2                      -1.249243           -1.248828   
2020-04-15 0-0.1                    -8.798222           -7.131714   
           0.1-0.4                  -2.081960           -1.774329   
           0.4-1                    -2.064796           -2.047637   
           1-2                      -1.248267           -1.248242   

                           soil_temp_min_cels  soil_moisture_avg_%  \
date       ground_depth_m                                            
2020-04-13 0-0.1                    -6.701929            36.912499   
           0.1-0.4                  -1.540000            35.874406   
           0.4-1                    -2.290977            26.103708   
           1-2                      -1.248242            19.343128   
2020-04-14 0-0.1                   -11.479395            37.454167   
           0.1-0.4                  -1.740742            35.620240   
           0.4-1                    -2.184303            26.103708   
           1-2                      -1.250000            19.343128   
2020-04-15 0-0.1                    -9.331738            37.816667   
           0.1-0.4                  -2.317500            35.466073   
           0.4-1                    -2.094303            26.103708   
           1-2                      -1.248633            19.343128   

                           soil_moisture_max_%  soil_moisture_min_%  \
date       ground_depth_m                                             
2020-04-13 0-0.1                     37.200001            36.633335   
           0.1-0.4                   36.011906            35.711906   
           0.4-1                     26.103708            26.103708   
           1-2                       19.343128            19.343128   
2020-04-14 0-0.1                     37.700001            37.200001   
           0.1-0.4                   35.711906            35.511906   
           0.4-1                     26.103708            26.103708   
           1-2                       19.343128            19.343128   
2020-04-15 0-0.1                     37.900002            37.700001   
           0.1-0.4                   35.511906            35.411907   
           0.4-1                     26.103708            26.103708   
           1-2                       19.343128            19.343128   

                                              geometry  
date       ground_depth_m                               
2020-04-13 0-0.1           POINT (-105.64822 40.31324)  
           0.1-0.4         POINT (-105.64822 40.31324)  
           0.4-1           POINT (-105.64822 40.31324)  
           1-2             POINT (-105.64822 40.31324)  
2020-04-14 0-0.1           POINT (-105.64822 40.31324)  
           0.1-0.4         POINT (-105.64822 40.31324)  
           0.4-1           POINT (-105.64822 40.31324)  
           1-2             POINT (-105.64822 40.31324)  
2020-04-15 0-0.1           POINT (-105.64822 40.31324)  
           0.1-0.4         POINT (-105.64822 40.31324)  
           0.4-1           POINT (-105.64822 40.31324)  
           1-2             POINT (-105.64822 40.31324)

Is there a way to combine the WeatherLocationForecast & WeatherLocationForecastSoil? The coord_cols, drop_cols, and rename_map are different, the get_data method is the same, and the extract data could be modified so that it returns two dataframes, one for main and one for soil.

Is there a way to overwrite coord cols so that the user can still do WeatherLocationForecast.coord_cols?

Add another method called extract_soil(), and define coord_cols and soil_coord cols

## Sub-sub-sub-class - WeatherLocationForecastSoil

In [230]:
class WeatherLocationForecastSoil(WeatherLocation):

    # Class variables for clean_data() function
    coord_cols = ['longitude', 'latitude']

    drop_cols = ['units']

    rename_map = {
        'average_temp': 'soil_temp_avg_cels',
        'max_temp': 'soil_temp_max_cels',
        'min_temp': 'soil_temp_min_cels',
        'average_moisture': 'soil_moisture_avg_%',
        'max_moisture': 'soil_moisture_max_%',
        'min_moisture': 'soil_moisture_min_%',
    }

    def __init__(self, api_key, api_secret, latitude, longitude, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(WeatherLocationForecastSoil, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude 
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/forecasts"

    def get_data(self, latitude, longitude, start_day=None, end_day=None, offset=0, block_size=24):
        """
        Performs a HTTP GET request to obtain the 7-day forecast.

        Docs:
            http://developer.awhere.com/api/forecast-weather-api

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response: dict
            Dictionary containing the forecast.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self.api_url}?limit=10&offset={offset}&blockSize={block_size}"
        url_start_date = f"{self.api_url}/{start_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_end_date = f"{self.api_url}/{end_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_both_dates = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}&blockSize={block_size}"

        # Perform the HTTP request to obtain the Forecast for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return forecast
        return response.json()
    
    @staticmethod
    def extract_data(forecast):
        """Extract aWhere forecast soil
        data and returns it in a pandas dataframe.
        """
        # Initialize lists to store soil dataframes
        forecast_soil_list = []

        # Check if more than one day
        if forecast.get('forecasts'):
            forecast_iterator = json_normalize(forecast.get('forecasts'))

        # Single day
        else:
            forecast_iterator = json_normalize(forecast)

        # Loop through each row in the top-level flattened dataframe
        for index, row in forecast_iterator.iterrows():

            # Extract date, lat, lon for insertion into lower-level dataframe outputs
            date = row['date']
            lat = row['location.latitude']
            lon = row['location.longitude']

            # Get soil temperature data
            forecast_soil_temp = row['forecast'][0].get('soilTemperatures')
            forecast_soil_moisture = row['forecast'][0].get('soilMoisture')

            # Flatten data into dataframe
            forecast_soil_temp_df = json_normalize(forecast_soil_temp)
            forecast_soil_moisture_df = json_normalize(forecast_soil_moisture)

            # Combine temperature and moisture
            forecast_soil_df = forecast_soil_temp_df.merge(
                forecast_soil_moisture_df, on='depth', suffixes=('_temp', '_moisture'))

            # Assign date, lat/lon to dataframe
            forecast_soil_df['date'] = date
            forecast_soil_df['latitude'] = lat
            forecast_soil_df['longitude'] = lon

            # Shorten depth values to numerics (will be used in MultiIndex)
            forecast_soil_df['depth'] = forecast_soil_df['depth'].apply(lambda x: x[0:-15])

            # Rename depth prior to indexing
            forecast_soil_df.rename(columns={'depth': 'ground_depth_m'}, inplace=True)

            # Create multi-index dataframe for date and soil depth (rename depth columns? rather long)
            soil_multi_index = forecast_soil_df.set_index(
                ['date', 'ground_depth_m'])

            # Add dataframe to list of dataframes
            forecast_soil_list.append(soil_multi_index)

        # Return merged lists of dataframes into a single dataframe
        return pd.concat(forecast_soil_list)

## Sub-sub-sub-class - WeatherNormsField

In [22]:
# Sub-sub-class - make one each for forecast, norms, and observed? Then sub-sub-sub-class for soil temp and soil moisture?
class WeatherNormsField(WeatherNorms):

    # Class variables for clean_data() function
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'location.fieldId', 'meanTemp.units', 'maxTemp.units',
        'minTemp.units', 'precipitation.units', 'solar.units',
        'dailyMaxWind.units', 'averageWind.units'
    ]

    rename_map = {
        'meanTemp.average': 'mean_temp_avg_cels',
        'meanTemp.stdDev': 'mean_temp_std_dev_cels',
        'maxTemp.average': 'max_temp_avg_cels',
        'maxTemp.stdDev': 'max_temp_std_dev_cels',
        'minTemp.average': 'min_temp_avg_cels',
        'minTemp.stdDev': 'min_temp_std_dev_cels',
        'precipitation.average': 'precip_avg_mm',
        'precipitation.stdDev': 'precip_std_dev_mm',
        'solar.average': 'solar_avg_w_h_per_m2',
        'minHumidity.average': 'min_humiduty_avg_%',
        'minHumidity.stdDev': 'min_humidity_std_dev_%',
        'maxHumidity.average': 'max_humiduty_avg_%',
        'maxHumidity.stdDev': 'max_humidity_std_dev_%',
        'dailyMaxWind.average': 'daily_max_wind_avg_m_per_sec',
        'dailyMaxWind.stdDev': 'daily_max_wind_std_dev_m_per_sec',
        'averageWind.average': 'average_wind_m_per_sec',
        'averageWind.stdDev': 'average_wind_std_dev_m_per_sec'
    }


    # ASSUMES A FIELD HAS ALREADY BEEN CREATED
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, _api_url=None):

        super(WeatherNormsField, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token)
        
        self.field_id = field_id
        self._api_url = 'https://api.awhere.com/v2/weather/fields'

    def get_data(self, field_id, start_day, end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 10-year historical norms.

        Docs:
            http://developer.awhere.com/api/reference/weather/norms

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the norms.

        Example
        -------
        """
        """# Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        response = requests.get(
            f"{self._weather_url}/{field_id}/norms/{start_day}",
            headers=auth_headers)

        # Return the norms
        return response.json()"""

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self._api_url}/{field_id}/norms/{start_day}?limit=10&offset={offset}"
        url_multiple_days = f"{self._api_url}/{field_id}/norms/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(historic_norms):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere historic norm data.

        Handles both single-day norms and multiple days.

        Parameters
        ----------
        historic_norms : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        historic_norms_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in norms
        if historic_norms.get('norms'):
            # Flatten to dataframe
            historic_norms_df = json_normalize(historic_norms.get('norms'))

        # Single-day norm
        else:
            # Flatten to dataframe
            historic_norms_df = json_normalize(historic_norms)

        # Set day as index
        historic_norms_df.set_index('day', inplace=True)

        # Drop unnecessary columns
        historic_norms_df.drop(
            columns=[
                '_links.self.href',
                '_links.curies',
                '_links.awhere:field.href'],
            axis=1, inplace=True)

        # Return dataframe
        return historic_norms_df


In [23]:
test_weather_norms = WeatherNormsField(api_key, api_secret, 'Colorado-Test-1')

In [24]:
norms_json = test_weather_norms.get_data(
    test_weather_norms.field_id, '05-05', '05-09')

In [152]:
norms_json

{'norms': [{'day': '05-05',
   'location': {'latitude': 40.0,
    'longitude': -105.0,
    'fieldId': 'Colorado-Test-1'},
   'meanTemp': {'average': 13.945999857783317,
    'stdDev': 3.8903210502507397,
    'units': 'C'},
   'maxTemp': {'average': 22.082999801635744,
    'stdDev': 5.551672393197332,
    'units': 'C'},
   'minTemp': {'average': 5.808999913930893,
    'stdDev': 3.525650157734424,
    'units': 'C'},
   'precipitation': {'average': 2.6057781100273134,
    'stdDev': 4.210783503304808,
    'units': 'mm'},
   'solar': {'average': 6363.582080078125,
    'stdDev': 1534.9628771050964,
    'units': 'Wh/m^2'},
   'minHumidity': {'average': 22.45399956703186, 'stdDev': 22.33335828373536},
   'maxHumidity': {'average': 70.45099983215331, 'stdDev': 13.848073684037963},
   'dailyMaxWind': {'average': 6.584204530715942,
    'stdDev': 2.1243740934866984,
    'units': 'm/sec'},
   'averageWind': {'average': 2.7954545378685,
    'stdDev': 0.8925154999205189,
    'units': 'm/sec'},
   '_li

In [25]:
df = WeatherNormsField.extract_data(norms_json)

In [26]:
df

,location.latitude,location.longitude,location.fieldId,meanTemp.average,meanTemp.stdDev,meanTemp.units,maxTemp.average,maxTemp.stdDev,maxTemp.units,minTemp.average,...,minHumidity.average,minHumidity.stdDev,maxHumidity.average,maxHumidity.stdDev,dailyMaxWind.average,dailyMaxWind.stdDev,dailyMaxWind.units,averageWind.average,averageWind.stdDev,averageWind.units
day,,,,,,,,,,,,,,,,,,,,,
05-05,40.0,-105.0,Colorado-Test-1,13.9460,3.890321,C,22.083,5.551672,C,5.809,...,22.454000,22.333358,70.451000,13.848074,6.584205,2.124374,m/sec,2.795455,0.892515,m/sec
05-06,40.0,-105.0,Colorado-Test-1,14.4150,3.582551,C,21.849,5.164409,C,6.981,...,26.088000,14.494682,73.768000,18.025834,7.038779,1.919636,m/sec,2.866111,0.954224,m/sec
05-07,40.0,-105.0,Colorado-Test-1,12.3885,4.763780,C,18.574,6.989987,C,6.203,...,39.069000,24.087735,84.339999,12.904113,7.047276,2.314907,m/sec,2.817376,1.084698,m/sec
05-08,40.0,-105.0,Colorado-Test-1,11.2900,4.472330,C,17.520,6.252500,C,5.060,...,42.004001,24.428537,84.154999,18.977546,6.414071,1.628986,m/sec,2.930409,0.871596,m/sec
05-09,40.0,-105.0,Colorado-Test-1,11.8510,5.320620,C,18.739,7.408928,C,4.963,...,38.504000,24.690818,81.045999,21.408004,7.503801,2.915196,m/sec,3.022484,1.338749,m/sec


In [27]:
gdf = Weather.clean_data(
    df,
    WeatherNormsField.coord_cols,
    WeatherNormsField.drop_cols,
    WeatherNormsField.rename_map
)

In [28]:
gdf

,mean_temp_avg_cels,mean_temp_std_dev_cels,max_temp_avg_cels,max_temp_std_dev_cels,min_temp_avg_cels,min_temp_std_dev_cels,precip_avg_mm,precip_std_dev_mm,solar_avg_w_h_per_m2,solar.stdDev,min_humiduty_avg_%,min_humidity_std_dev_%,max_humiduty_avg_%,max_humidity_std_dev_%,daily_max_wind_avg_m_per_sec,daily_max_wind_std_dev_m_per_sec,average_wind_m_per_sec,average_wind_std_dev_m_per_sec,geometry
day,,,,,,,,,,,,,,,,,,,
05-05,13.9460,3.890321,22.083,5.551672,5.809,3.525650,2.605778,4.210784,6363.582080,1534.962877,22.454000,22.333358,70.451000,13.848074,6.584205,2.124374,2.795455,0.892515,POINT (-105.00000 40.00000)
05-06,14.4150,3.582551,21.849,5.164409,6.981,2.498215,4.613071,7.772575,5644.947485,1102.703641,26.088000,14.494682,73.768000,18.025834,7.038779,1.919636,2.866111,0.954224,POINT (-105.00000 40.00000)
05-07,12.3885,4.763780,18.574,6.989987,6.203,3.236504,6.720669,4.666867,4555.624194,1505.715592,39.069000,24.087735,84.339999,12.904113,7.047276,2.314907,2.817376,1.084698,POINT (-105.00000 40.00000)
05-08,11.2900,4.472330,17.520,6.252500,5.060,3.502110,5.545790,6.694246,4932.714453,2058.613223,42.004001,24.428537,84.154999,18.977546,6.414071,1.628986,2.930409,0.871596,POINT (-105.00000 40.00000)
05-09,11.8510,5.320620,18.739,7.408928,4.963,4.130421,7.477840,10.010236,5319.268359,1856.340854,38.504000,24.690818,81.045999,21.408004,7.503801,2.915196,3.022484,1.338749,POINT (-105.00000 40.00000)


## Sub-sub-class - WeatherObserved

In [29]:
class WeatherObserved(Weather):

    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, latitude=0, longitude=0):

        super(WeatherObserved, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, latitude, longitude)

        #def get_observed():
         #   pass

## Sub-sub-sub-class - WeatherObservedLocation

In [30]:
class WeatherObservedLocation(WeatherObserved):

    # Class variables for clean_data() function
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units'
    ]

    rename_map = {
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.amount': 'precip_amount_mm',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.morningMax': 'wind_morning_max_m_per_sec',
        'wind.dayMax': 'wind_day_max_m_per_sec',
        'wind.average': 'wind_avg_m_per_sec',
    }

    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, latitude=0, longitude=0, _api_url=None):

        super(WeatherObservedLocation, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, latitude, longitude)

        self._api_url = 'https://api.awhere.com/v2/weather/locations'

    def get_data(self, latitude, longitude, start_day=None, end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 7-day observed weather.

        Docs:
            http://developer.awhere.com/api/reference/weather/observations

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the observed weather.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self._api_url}/{latitude},{longitude}/observations?limit=10&offset={offset}"
        url_start_date = f"{self._api_url}/{latitude},{longitude}/observations/{start_day}"
        url_end_date = f"{self._api_url}/{latitude},{longitude}/observations/{end_day}"
        url_both_dates = f"{self._api_url}/{latitude},{longitude}/observations/{start_day},{end_day}?limit=10&offset={offset}"

        # Perform the HTTP request to obtain the norms for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return the observed
        return response.json()

    @staticmethod
    def extract_data(observed_weather):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere observed weather data.

        Parameters
        ----------
        observed_weather : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        observed_weather_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in observed
        if observed_weather.get('observations'):
            # Flatten to dataframe
            observed_weather_df = json_normalize(
                observed_weather.get('observations'))

        # Single-day observed
        else:
            # Flatten to dataframe
            observed_weather_df = json_normalize(observed_weather)

        # Set date as index
        observed_weather_df.set_index('date', inplace=True)

        # Drop unnecessary columns
        observed_weather_df.drop(
            columns=['_links.self.href'],
            axis=1, inplace=True)

        # Return dataframe
        return observed_weather_df

In [31]:
observed_loc_return = WeatherObservedLocation(api_key, api_secret, latitude=42.5, longitude=-72.5)

In [209]:
#observed_loc_return.get_observed(observed_loc_return.latitude, observed_loc_return.longitude)

In [33]:
df = WeatherObservedLocation.extract_data(observed_loc_return.get_data(
    observed_loc_return.latitude, observed_loc_return.longitude))

In [210]:
#df

In [35]:
gdf = Weather.clean_data(
    df,
    WeatherObservedLocation.coord_cols,
    WeatherObservedLocation.drop_cols,
    WeatherObservedLocation.rename_map
)

In [211]:
#gdf

## Sub-sub-sub-class - WeatherObservedField

In [37]:
class WeatherObservedField(WeatherObserved):

    # Class variables for clean_data() function  
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'location.fieldId', 'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units'
    ]

    rename_map = {
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.amount': 'precip_amount_mm',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.morningMax': 'wind_morning_max_m_per_sec',
        'wind.dayMax': 'wind_day_max_m_per_sec',
        'wind.average': 'wind_avg_m_per_sec',
    }

    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, _api_url=None):

        super(WeatherObservedField, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token)

        self.field_id = field_id
        self._api_url = 'https://api.awhere.com/v2/weather/fields'

    def get_data(self, field_id, start_day=None, end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 7-day observed weather.

        Docs:
            http://developer.awhere.com/api/reference/weather/observations

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the observed weather.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self._api_url}/{field_id}/observations?limit=10&offset={offset}"
        url_start_date = f"{self._api_url}/{field_id}/observations/{start_day}"
        url_end_date = f"{self._api_url}/{field_id}/observations/{end_day}"
        url_both_dates = f"{self._api_url}/{field_id}/observations/{start_day},{end_day}?limit=10&offset={offset}"

        # Perform the HTTP request to obtain the norms for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return the observed
        return response.json()

    @staticmethod
    def extract_data(observed_weather):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere observed weather data.

        Parameters
        ----------
        observed_weather : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        observed_weather_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in observed
        if observed_weather.get('observations'):
            # Flatten to dataframe
            observed_weather_df = json_normalize(observed_weather.get('observations'))

        # Single-day observed
        else:
            # Flatten to dataframe
            observed_weather_df = json_normalize(observed_weather)

        # Set date as index
        observed_weather_df.set_index('date', inplace=True)

        # Drop unnecessary columns
        observed_weather_df.drop(
            columns=[
                '_links.self.href',
                '_links.curies',
                '_links.awhere:field.href'],
            axis=1, inplace=True)

        # Return dataframe
        return observed_weather_df

In [38]:
observed_loc_return = WeatherObservedField(api_key, api_secret, 'Colorado-Test-1')

In [40]:
observed_loc_return.get_data(observed_loc_return.field_id)

{'observations': [{'date': '2020-04-04',
   'location': {'latitude': 40.0,
    'longitude': -105.0,
    'fieldId': 'Colorado-Test-1'},
   'temperatures': {'max': 16.020000457763672,
    'min': -3.4800000190734863,
    'units': 'C'},
   'precipitation': {'amount': 0.0, 'units': 'mm'},
   'solar': {'amount': 5447.1220703125, 'units': 'Wh/m^2'},
   'relativeHumidity': {'max': 77.4000015258789, 'min': 22.790000915527344},
   'wind': {'morningMax': 3.6919596195220947,
    'dayMax': 5.024531364440918,
    'average': 1.9949634075164795,
    'units': 'm/sec'},
   '_links': {'self': {'href': '/v2/weather/fields/Colorado-Test-1/observations/2020-04-04'},
    'curies': [{'name': 'awhere',
      'href': 'http://awhere.com/rels/{rel}',
      'templated': True}],
    'awhere:field': {'href': '/v2/fields/Colorado-Test-1'}}},
  {'date': '2020-04-05',
   'location': {'latitude': 40.0,
    'longitude': -105.0,
    'fieldId': 'Colorado-Test-1'},
   'temperatures': {'max': 21.469999313354492,
    'min': 0

In [41]:
df = WeatherObservedField.extract_data(observed_loc_return.get_data(observed_loc_return.field_id))

In [42]:
df

,location.latitude,location.longitude,location.fieldId,temperatures.max,temperatures.min,temperatures.units,precipitation.amount,precipitation.units,solar.amount,solar.units,relativeHumidity.max,relativeHumidity.min,wind.morningMax,wind.dayMax,wind.average,wind.units
date,,,,,,,,,,,,,,,,
2020-04-04,40.0,-105.0,Colorado-Test-1,16.020000,-3.48,C,0.0,mm,5447.122070,Wh/m^2,77.400002,22.790001,3.691960,5.024531,1.994963,m/sec
2020-04-05,40.0,-105.0,Colorado-Test-1,21.469999,0.58,C,0.0,mm,5236.607910,Wh/m^2,77.809998,16.740000,3.382703,4.784543,1.873570,m/sec
2020-04-06,40.0,-105.0,Colorado-Test-1,23.090000,3.14,C,0.0,mm,6616.105957,Wh/m^2,55.000000,9.470000,4.802201,5.575871,2.486083,m/sec
2020-04-07,40.0,-105.0,Colorado-Test-1,23.190001,5.23,C,0.0,mm,6680.076172,Wh/m^2,39.180000,9.710000,7.915365,10.256870,5.143219,m/sec
2020-04-08,40.0,-105.0,Colorado-Test-1,21.530001,3.34,C,0.0,mm,6473.966309,Wh/m^2,60.380001,16.010000,5.158222,6.507509,2.459224,m/sec
2020-04-09,40.0,-105.0,Colorado-Test-1,14.850000,0.56,C,0.0,mm,5993.878906,Wh/m^2,83.589996,36.779999,5.781534,6.038283,2.697900,m/sec
2020-04-10,40.0,-105.0,Colorado-Test-1,22.549999,2.75,C,0.0,mm,5855.743652,Wh/m^2,77.570000,13.960000,4.562067,8.876784,2.869099,m/sec


In [43]:
# Can either use Weather.clean_data() or WeatherObservedField.clean_data()
# Function is defined at higher level, inherited at lower level
gdf = WeatherObservedField.clean_data(
    df,
    WeatherObservedField.coord_cols,
    WeatherObservedField.drop_cols,
    WeatherObservedField.rename_map
)

In [44]:
gdf

,temp_max_cels,temp_min_cels,precip_amount_mm,solar_energy_w_h_per_m2,rel_humidity_max_%,rel_humidity_min_%,wind_morning_max_m_per_sec,wind_day_max_m_per_sec,wind_avg_m_per_sec,geometry
date,,,,,,,,,,
2020-04-04,16.020000,-3.48,0.0,5447.122070,77.400002,22.790001,3.691960,5.024531,1.994963,POINT (-105.00000 40.00000)
2020-04-05,21.469999,0.58,0.0,5236.607910,77.809998,16.740000,3.382703,4.784543,1.873570,POINT (-105.00000 40.00000)
2020-04-06,23.090000,3.14,0.0,6616.105957,55.000000,9.470000,4.802201,5.575871,2.486083,POINT (-105.00000 40.00000)
2020-04-07,23.190001,5.23,0.0,6680.076172,39.180000,9.710000,7.915365,10.256870,5.143219,POINT (-105.00000 40.00000)
2020-04-08,21.530001,3.34,0.0,6473.966309,60.380001,16.010000,5.158222,6.507509,2.459224,POINT (-105.00000 40.00000)
2020-04-09,14.850000,0.56,0.0,5993.878906,83.589996,36.779999,5.781534,6.038283,2.697900,POINT (-105.00000 40.00000)
2020-04-10,22.549999,2.75,0.0,5855.743652,77.570000,13.960000,4.562067,8.876784,2.869099,POINT (-105.00000 40.00000)


In [ ]:
# Create a function that takes the AWhere Class Object and
#  computes json return data, df, and gdf, and outputs gdf.
# This would be a simple call for the user.

In [71]:
# Sub-sub-class - make one each for forecast, norms, and observed? Then sub-sub-sub-class for soil temp and soil moisture?
class WeatherLocation(Weather):

#     # Class variables - # Make this name more generic - for use with other classes lon_lat_cols,
#     # drop_cols, mapping, etc.
#     lon_lat_cols = ['longitude', 'latitude']

#     drop_cols = [
#         'temperatures.units', 'precipitation.units',
#         'solar.units', 'wind.units', 'dewPoint.units'
#     ]

#     mapping = {
#         'startTime': 'start_time',
#         'endTime': 'end_time',
#         'conditionsCode': 'conditions_code',
#         'conditionsText': 'conditions_text',
#         'temperatures.max': 'temp_max_cels',
#         'temperatures.min': 'temp_min_cels',
#         'precipitation.chance': 'precip_chance_%',
#         'precipitation.amount': 'precip_amount_mm',
#         'sky.cloudCover': 'sky_cloud_cover_%',
#         'sky.sunshine': 'sky_sunshine_%',
#         'solar.amount': 'solar_energy_w_h_per_m2',
#         'relativeHumidity.average': 'rel_humidity_avg_%',
#         'relativeHumidity.max': 'rel_humidity_max_%',
#         'relativeHumidity.min': 'rel_humidity_min_%',
#         'wind.average': 'wind_avg_m_per_sec',
#         'wind.max': 'wind_max_m_per_sec',
#         'wind.min': 'wind_min_m_per_sec',
#         'wind.bearing': 'wind_bearing_deg',
#         'wind.direction': 'wind_direction_compass',
#         'dewPoint.amount': 'dew_point_cels'
#     }

    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, latitude=0, longitude=0, _api_url=None):

        super(WeatherLocation, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, latitude, longitude)

        self._api_url = 'https://api.awhere.com/v2/weather/locations'

#     def get_weather_forecast(self, latitude, longitude, start_day=None,
#                              end_day=None, offset=0, block_size=24):
#         """
#         Performs a HTTP GET request to obtain the 7-day forecast.

#         Docs:
#             http://developer.awhere.com/api/forecast-weather-api

#         Parameters
#         ----------
#         field_id : str
#             ID of the field.

#         Returns
#         -------
#         response: dict
#             Dictionary containing the forecast.

#         Example
#         -------
#         """
#         # Setup the HTTP request headers
#         auth_headers = {
#             "Authorization": f"Bearer {self.auth_token}"
#         }

#         # Define URL variants
#         url_no_date = f"{self._geolocation_url}/{latitude},{longitude}/forecasts?limit=10&offset={offset}&blockSize={block_size}"
#         url_start_date = f"{self._geolocation_url}/{latitude},{longitude}/forecasts/{start_day}?limit=10&offset={offset}&blockSize={block_size}"
#         url_end_date = f"{self._geolocation_url}/{latitude},{longitude}/forecasts/{end_day}?limit=10&offset={offset}&blockSize={block_size}"
#         url_both_dates = f"{self._geolocation_url}/{latitude},{longitude}/forecasts/{start_day},{end_day}?limit=10&offset={offset}&blockSize={block_size}"

#         # Perform the HTTP request to obtain the Forecast for the Field
#         # Default - 7-day
#         if not (start_day or end_day):
#             response = requests.get(url_no_date, headers=auth_headers)

#         # Single date - specify start day
#         elif start_day and not end_day:
#             response = requests.get(url_start_date, headers=auth_headers)

#         # Single date - specify end day
#         elif end_day and not start_day:
#             response = requests.get(url_end_date, headers=auth_headers)

#         # Date range
#         elif start_day and end_day:
#             response = requests.get(url_both_dates, headers=auth_headers)

#         # Return forecast
#         return response.json()

#     # Make this name more generic - for use with other classes (Class.extract())
#     @staticmethod
#     def extract_forecast_main_data(forecast):
#         """Extract aWhere forecast data and returns
#         it in a pandas dataframe.
#         """
#         # Initialize lists to store forecast
#         forecast_main_list = []

#         # Check if more than one day
#         if forecast.get('forecasts'):
#             forecast_iterator = json_normalize(forecast.get('forecasts'))

#         # Single day
#         else:
#             forecast_iterator = json_normalize(forecast)

#         # Loop through each row in the top-level flattened dataframe
#         for index, row in forecast_iterator.iterrows():

#             # Extract date, lat, lon for insertion into lower-level dataframe outputs
#             date = row['date']
#             lat = row['location.latitude']
#             lon = row['location.longitude']

#             # Extract the main forecast from the top-level dataframe
#             forecast = row['forecast']

#             # Faltten data into dataframe
#             forecast_norm = json_normalize(forecast)

#             # Drop soil moisture and soil temperature columns
#             #  (will be extracted as indivdiual dataframes)
#             forecast_norm.drop(columns=[
#                 'soilTemperatures',
#                 'soilMoisture',
#             ],
#                 axis=1, inplace=True)
#             # Assign date, lat/lon to dataframe
#             forecast_norm['date'] = date
#             forecast_norm['latitude'] = lat
#             forecast_norm['longitude'] = lon

#             # Set date as index
#             forecast_norm.set_index(['date'], inplace=True)

#             # Add the dataframe to a list of dataframes
#             forecast_main_list.append(forecast_norm)

#         # Return merged lists of dataframes into a single dataframe
#         return pd.concat(forecast_main_list)

#     # Make this name more generic - for use with other classes (Class.clean())
#     @staticmethod
#     def clean_dataframe(df, lon_lat_cols, drop_cols, name_map):
#         """Converts dataframe to geodataframe,
#         drops unnecessary columns, and renames
#         columns.

#         Parameters
#         ----------
#         df : dataframe
#             Input dataframe.

#         lon_lat_cols : list
#             List containing the column name for longitude (list[0])
#             and latitude (list[1]) attributes.

#         drop_cols : list (of str)
#             List of column names to be dropped.

#         name_map : dict
#             Dictionaty mapping old columns names (keys)
#             to new column names (values).

#         Returns
#         -------
#         gdf : geodataframe
#             Cleaned geodataframe.

#         Example
#         -------
#         """
#         # Define CRS (EPSG 4326) - make this a parameter?
#         crs = {'init': 'epsg:4326'}

#         # Rename index - possibly as option, or take care of index prior?
#         #df.index.rename('date_rename', inplace=True)

#         # Create copy of input dataframe; prevents altering the original
#         df_copy = df.copy()

#         # Convert to geodataframe
#         gdf = gpd.GeoDataFrame(
#             df_copy, crs=crs, geometry=gpd.points_from_xy(
#                 df[lon_lat_cols[0]],
#                 df[lon_lat_cols[1]])
#         )

#         # Add lat/lon columns to drop columns list
#         drop_cols += lon_lat_cols

#         # Drop columns
#         gdf.drop(columns=drop_cols, axis=1, inplace=True)

#         # Rename columns
#         gdf.rename(columns=name_map, inplace=True)

#         # Return cleaned up geodataframe
#         return gdf

## Sub-sub-class - WeatherField

In [ ]:
# Sub-sub-class - make one each for forecast, norms, and observed? Then sub-sub-sub-class for soil temp and soil moisture?
class WeatherField(Weather):

    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None,
                 auth_token=None, latitude=0, longitude=0, _api_url=None):

        super(WeatherField, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, latitude, longitude)

        self._api_url = 'https://api.awhere.com/v2/weather/fields'